In [14]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import Dataset, DataLoader 

In [2]:
fpath='/its/home/gc349/Causal_inf_convattention/data'
fname='/generated_var.csv'
df=pd.read_csv(fpath+fname)

In [3]:
#model MLP with arbitrary number of hidden layers
class MLP(torch.nn.Module):
    def __init__(self, h_sizes, out_size):

        super(MLP, self).__init__()

        # Hidden layers
        self.hidden = nn.ModuleList()
        for k in range(len(h_sizes)-1):
            self.hidden.append(nn.Linear(h_sizes[k], h_sizes[k+1]))
        # Output layer
        self.out = nn.Linear(h_sizes[-1], out_size)

    def forward(self, x):

        # Feedforward
        for layer in self.hidden:
            x = F.relu(layer(x))
        output= self.out(x)
        return output

In [4]:
#transform to supervised learning pb
def timeseries_to_supervised(data, lag=1):
    df = pd.DataFrame(data)
    columns = [df.shift(i) for i in range(1, lag+1)]
    columns.append(df)
    df = pd.concat(columns, axis=1)
    df.dropna(inplace=True)
    return df

In [5]:
train_size=int(0.75*len(df.index))
test_size=int(0.25*len(df.index))
tsdata=timeseries_to_supervised(df,lag=1)
#create test, training set dataframe
df_X_train=tsdata.iloc[0:train_size, 0:2]
df_X_test=tsdata.iloc[-test_size:, 0:2]
df_target_train=tsdata.iloc[0:train_size, 2:4]
df_target_test=tsdata.iloc[-test_size:, 2:4]
#Transform them in arrays
X_train=df_X_train.values
X_test=df_X_test.values
target_train=df_target_train.values
target_test=df_target_test.values

In [6]:
# scale train and test data to [-1, 1]
def scale(train, test):
	# fit scaler
	scaler = MinMaxScaler(feature_range=(-1, 1))
	scaler = scaler.fit(train)
	# transform train
	train = train.reshape(train.shape[0], train.shape[1])
	train_scaled = scaler.transform(train)
	# transform test
	test = test.reshape(test.shape[0], test.shape[1])
	test_scaled = scaler.transform(test)
	return scaler, train_scaled, test_scaled

In [7]:
_, Xsc_train, Xsc_test=scale(X_train, X_test)
#transform them in tensor
X_train_tens=torch.tensor(Xsc_train)
X_test_tens=torch.tensor(Xsc_test)
target_train_tens=torch.tensor(df_target_train.values)
target_test_tens=torch.tensor(df_target_test.values)
h_sizes=[np.size(Xsc_train,1), 4, 8, 8]
out_size=np.size(target_train,1)

In [12]:
model=MLP(h_sizes, out_size)

In [15]:
# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the two
# nn.Linear modules which are members of the model.
#Problem might be with number of features in and out
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
for t in range(500):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(X_train_tens)

    # Compute and print loss
    loss = criterion(y_pred, target_train_tens)
    print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

RuntimeError: Expected object of scalar type Float but got scalar type Double for argument #4 'mat1'